In [40]:
from pandas import *
import json
from requests import *
from numpy import *
from toucan_data_sdk.utils.generic import *

In [2]:
data_input = read_csv("../Agenda24072015.csv", sep=";", low_memory=False)
# populations = read_excel('../base-ic-evol-struct-pop-2015.xls', skiprows=5, usecols=['COM', 'P15_POP'])

In [3]:
data_input.columns

Index(['idMemo', 'libelleOrg', 'ancienNom', 'typeOrganisme', 'natureOrganisme',
       'adresseComplAdresseOrg', 'adresseVoieNumeroOrg',
       'adresseVoieExtOuIndRepOrg', 'adresseVoieTypeVoieOrg',
       'adresseVoieNomVoieOrg', 'adresseMentionsDistribOrg',
       'adresseAcheminPostalCedexOrg', 'adresseAcheminPostalLibelleCedexOrg',
       'adresseCodePostalOrg', 'adresseCommuneOrg', 'adresseCodeInseeOrg',
       'regionOrg', 'paysOrg', 'latitudeOrg', 'longitudeOrg', 'libelleOffre',
       'typeOffre', 'themes', 'periodeDateDebut', 'periodeDateFin',
       'hebdoEstOuvertLundi', 'hebdoEstOuvertMardi', 'hebdoEstOuvertMercredi',
       'hebdoEstOuvertJeudi', 'hebdoEstOuvertVendredi', 'hebdoEstOuvertSamedi',
       'hebdoEstOuvertDimanche', 'horaire', 'conditions', 'tarifs', 'gratuit',
       'libelleLieu', 'adresseComplAdresseLieu', 'adresseVoieNumeroLieu',
       'adresseVoieExtOuIndRepLieu', 'adresseVoieTypeVoieLieu',
       'adresseVoieNomVoieLieu', 'adresseMentionsDistribLieu',
  

In [68]:
from pandas import *
from numpy import *

offres = {
    'SPECTACLE':'SPECTACLE',
     'FESTIVAL':'MUSIQUE',
     'MUSÉE':'MUSEE',
     'EXPOSITION':"ART",
     'VISITE':"MUSEE",
     'ATELIER':"ATELIER",
     'Conférence':"SCIENCE",
     'Circuit':"ART",
     'Portes ouvertes':"MUSEE",
     'PROJECTION':"SPECTACLE",
     'LECTURE':"LECTURE",
     'SALON':"ART"
}


MUSEE = [
    "Architecture"
]

LECTURE = ["Livre"]
 
MUSIQUE = [
#     "MUSIQUE",
"Classique et contemporaine",
"Jazz",
"Variété",
"Musiques du monde",
"Opéra",
"Baroque",
"Pop, Rock, Folk",
"Lyrique et chant choral",
"Folkloriques ou traditionnelles", 
"Classique et contemporaine",
"Électronique",
"Opérette, Opéra-bouffe",
"Sacrée",
"Rap",
"Blues"]
#          "ART",
ART = [
"Photographie",
"Arts graphiques",
"Peinture",
"Beaux-Arts",
"Arts décoratifs",
"Art contemporain",
"Mode et textiles",
"Arts de la rue",
"Multimédia",
"Design",
"Arts plastiques"]

#          "SCIENCE",
SCIENCE = [
    "Sciences humaines","Sciences de la nature","Études et recherches culturelles"]
#          "SPECTACLE",

SPECTACLE = [
    "Cinéma","Ciné-concert", 
    "Comédie musicale", "Humour", "Théâtre", "cirque", "Danse", "Jeune Public", 
    "Cabaret", "Son et lumières"]

subtheme = MUSIQUE+ART+SCIENCE+SPECTACLE+MUSEE+LECTURE

d = {}
for a in ["MUSIQUE", "ART", "SCIENCE", "SPECTACLE", 'MUSEE']:
    for i in globals()[a]:
        d[i.upper()] =a 


def extract_info(data, column, info_to_extract):
    number_of_info_extracted = Series(full(data.shape[0], 0))
    info_extracted = Series(full(data.shape[0], ""))
    column_str  = data[column].str.upper().str
    
    for t in info_to_extract:
        t = t.upper()
        info_extracted+=(number_of_info_extracted==0)*column_str.find(t).apply(lambda x: (x>-1)*t)
        number_of_info_extracted+=(column_str.find(t)>-1)
    
    info_extracted = info_extracted.replace("", NaN)
    return info_extracted
    
    
def prepare_data(data):
#     pop = (pop.groupby(['COM']).sum().reset_index().rename(columns={"COM": "CODE_POSTAL", "P15_POP": "POPULATION"}))

    data = data.assign(**{
        "REGION":lambda x: x['regionLieu'],
        "COMMUNE":lambda x: x['adresseCommuneLieu'].str.capitalize(),
        "CODE_POSTAL":lambda x: x['adresseCodeInseeLieu'],
        "DEP_CODE": lambda x: x["adresseCodePostalLieu"].str[:2],
        "SUBTHEME":lambda x: extract_info(data, "themes", subtheme),
        "THEME":lambda x: x["SUBTHEME"].replace(d),
        "TYPE":lambda x: (extract_info(data, "typeOffre", list(offres.keys()))
                          .replace(offres)),
        "DUREE": lambda x: ((to_datetime(x.periodeDateFin, format='%d/%m/%Y', errors='coerce')-
                            to_datetime(x.periodeDateDebut, format='%d/%m/%Y', errors='coerce'))
                            .dt.days.clip(1, 2*365))
    })
    
    data["THEME"] = data.THEME.fillna(data.TYPE)
    data["SUBTHEME"] = data.THEME+" > "+data.SUBTHEME.fillna("AUTRES")
    data = data[['REGION', 'COMMUNE', 'THEME', 'SUBTHEME', 'TYPE', 'DUREE', 'CODE_POSTAL', 'DEP_CODE']]
    
#     a = data.shape[0]
#     data = data.merge(pop, on='CODE_POSTAL')
#     assert a == data.shape[0], f"Le nombre de ligne n'a pas ete conserve : avant : {a} apres : {data.shape[0]} " 
    data["VALUE"]=1
    return data

In [69]:
data = prepare_data(data_input)


In [70]:
data

,REGION,COMMUNE,THEME,SUBTHEME,TYPE,DUREE,CODE_POSTAL,DEP_CODE,VALUE
0,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,SPECTACLE,12.0,93066,93,1
1,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,SPECTACLE,8.0,93066,93,1
2,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,SPECTACLE,1.0,93066,93,1
3,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,MUSIQUE,12.0,93066,93,1
4,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,SPECTACLE,3.0,93066,93,1
5,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,SPECTACLE,8.0,93066,93,1
6,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,SPECTACLE,21.0,93066,93,1
7,Bourgogne,Louhans,MUSIQUE,MUSIQUE > RAP,MUSEE,NaN,71263,71,1
8,Bourgogne,Verdun-sur-le-doubs,MUSEE,MUSEE > AUTRES,MUSEE,NaN,71566,71,1
9,Ile-de-France,Milly-la-forêt,ART,ART > AUTRES,ART,1.0,91405,91,1


In [66]:
a = get("https://raw.githubusercontent.com/rhuille/files/master/basemap/france_departement.json").json()
a = {i["properties"]["code"]:i["properties"]["nom"] for i in a["features"]}
(data
 .groupby("DEP_CODE").VALUE.sum().reset_index()
 .assign(LABEL = lambda x: x["DEP_CODE"].replace(a))
).to_csv('data_map.csv', index=False)

In [76]:
(DataFrame({
    "SUBTHEME":['CHOISISSEZ VOTRE ACTIVITE FAVORITE ;)']+list(data.sort_values(by="SUBTHEME").SUBTHEME.unique())})
#  .sort_values(by="SUBTHEME")
 .to_csv("subtheme_requester.csv", index=False)
)

In [37]:
(DataFrame({"COMMUNE":data.COMMUNE.unique()})
 .sort_values(by="COMMUNE")
 .to_csv("reports.csv", index=False)
)

In [67]:
data_aggregated = roll_up(data,
                          levels=["COMMUNE", "SUBTHEME"],
                          groupby_vars=["VALUE"])

data_aggregated = data_aggregated[["COMMUNE", "SUBTHEME", "VALUE", "type"]]

# data_aggregated.to_csv('data_aggregated.csv', index=False)

In [82]:
data_aggregated_theme = roll_up(data,
                          levels=["COMMUNE", "THEME"],
                          groupby_vars=["VALUE"])

data_aggregated_theme = data_aggregated_theme[["COMMUNE", "THEME", "VALUE", "type"]]

data_aggregated_theme = data.g
data_aggregated_theme

,COMMUNE,THEME,VALUE,type
0,médis,MUSIQUE,1,THEME
1,Abbeville,MUSIQUE,1,THEME
2,Abilly,ART,1,THEME
3,Ablon-sur-seine,ART,1,THEME
4,Agde,ART,1,THEME
5,Agde,MUSEE,1,THEME
6,Agde,SPECTACLE,1,THEME
7,Agen,ART,5,THEME
8,Agen,MUSEE,3,THEME
9,Agen,MUSIQUE,1,THEME


In [86]:
d = data.groupby("THEME").VALUE.sum().reset_index()

In [87]:
d

,THEME,VALUE
0,ART,5013
1,ATELIER,344
2,CIRCUIT,91
3,CONFÉRENCE,481
4,LECTURE,80
5,LIVRE,719
6,MUSEE,2341
7,MUSIQUE,10488
8,PORTES OUVERTES,66
9,SCIENCE,611


In [ ]:
roll_up(data,levels=["COMMUNE", "SUBTHEME"],
                          groupby_vars=["VALUE"])

In [32]:
data

,REGION,COMMUNE,THEME,SUBTHEME,TYPE,DUREE,CODE_POSTAL,VALUE,RANK
0,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,SPECTACLE,12.0,93066,1,5387.0
1,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,SPECTACLE,8.0,93066,1,5387.0
2,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,SPECTACLE,1.0,93066,1,5387.0
3,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,FESTIVAL,12.0,93066,1,5387.0
4,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,SPECTACLE,3.0,93066,1,5387.0
5,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,SPECTACLE,8.0,93066,1,5387.0
6,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,SPECTACLE,21.0,93066,1,5387.0
7,Bourgogne,Louhans,ART,RAP,MUSÉE,NaN,71263,1,5387.0
8,Bourgogne,Verdun-sur-le-Doubs,SCIENCE,,MUSÉE,NaN,71566,1,5387.0
9,Ile-de-France,Milly-la-Forêt,,,EXPOSITION,1.0,91405,1,5387.0


In [31]:
data_aggregated

,COMMUNE,SUBTHEME,VALUE,type,RANK
0,Médis,,1,SUBTHEME,5387
1,AIME EN TARENTAISE,,1,SUBTHEME,5387
2,ALLEMONT,ART CONTEMPORAIN,1,SUBTHEME,5387
3,ALLEVARD LES BAINS,,1,SUBTHEME,5387
4,AMIENS,,1,SUBTHEME,5387
5,ANDLAU,ARTS PLASTIQUES,1,SUBTHEME,5387
6,ANNECY,SCIENCES DE LA NATURE,1,SUBTHEME,5387
7,ARROMANCHES,RAP,1,SUBTHEME,5387
8,Abbeville,RAP,1,SUBTHEME,5387
9,Abilly,,1,SUBTHEME,5387


In [63]:
data.columns

Index(['REGION', 'COMMUNE', 'THEME', 'SUBTHEME', 'TYPE', 'DUREE',
       'CODE_POSTAL', 'POPULATION'],
      dtype='object')

In [76]:
(data
 .groupby(["COMMUNE", "THEME", 'SUBTHEME']).count()['REGION'].reset_index()
 .rename(columns={'REGION':'VALUE'})
 .to_csv('data_aggregated.csv', index=False))

In [85]:
data_aggregated_by_commune

,COMMUNE,THEME,SUBTHEME,VALUE
0,ANDLAU,ART,ARTS PLASTIQUES,1
1,ANNECY,SCIENCE,SCIENCES DE LA NATURE,1
2,Abbeville,ART,RAP,1
3,Ablon-sur-Seine,ART,ARTS DE LA RUE,1
4,Agde,ART,ARTS DÉCORATIFS,1
5,Agen,ART,PEINTURE,2
6,Agen,ART,RAP,1
7,Agen,SPECTACLE,THÉÂTRE,1
8,Aigues-Vives,MUSIQUE,CLASSIQUE ET CONTEMPORAINE,1
9,Ainay-le-Vieil,MUSIQUE,CLASSIQUE ET CONTEMPORAINE,1


In [ ]:
data

In [97]:
(data.COMMUNE.value_counts().reset_index()
 .rename(columns={'index':'Commune', 'COMMUNE':'Value'})
#  .to_csv('count_event_by_commune')
)

,Commune,Value
0,Paris 19ème,730
1,Paris 8ème,629
2,Rennes,540
3,Nantes,515
4,Toulouse,489
5,Dijon,481
6,Paris 16ème,408
7,Paris 1er,388
8,Paris 4ème,379
9,Montpellier,330


In [89]:
data.count()

REGION         33112
COMMUNE        33145
THEME          25332
SUBTHEME       22911
TYPE           32918
DUREE          31101
CODE_POSTAL    33145
POPULATION     33145
dtype: int64

In [101]:
data["VALUE"] = 1

In [102]:
data_aggregated_by_commune = (data
 .groupby(["COMMUNE", "THEME", 'SUBTHEME']).VALUE.sum().reset_index()
 .rename(columns={'REGION':'VALUE'}))

data_aggregated_by_commune.groupby("COMMUNE").sum().reset_index().query('COMMUNE=="Paris 19ème"')

,COMMUNE,VALUE
1361,Paris 19ème,730


In [3]:
def extract_info(data, column, info_to_extract):
    number_of_info_extracted = Series(full(data.shape[0], 0))
    info_extracted = Series(full(data.shape[0], ""))
    column_str  = data[column].str.upper().str
    
    for t in info_to_extract:
        t = t.upper()
        info_extracted+=(number_of_info_extracted==0)*column_str.find(t).apply(lambda x: (x>-1)*t)
        number_of_info_extracted+=(column_str.find(t)>-1)
    
    info_extracted = info_extracted.replace("", NAN)
    return info_extracted

In [103]:
data = read_csv("../Agenda24072015.csv", sep=";", low_memory=False)

In [104]:
pop = read_excel('../base-ic-evol-struct-pop-2015.xls', skiprows=5, usecols=['COM', 'P15_POP'])
pop = pop.groupby(['COM']).sum().reset_index().rename(columns={"COM": "CODE_POSTAL", "P15_POP": "POPULATION"})

In [105]:
data = data.assign(**{
    "REGION":lambda x: x['regionLieu'],
    "COMMUNE":lambda x: x['adresseCommuneLieu'],
    "CODE_POSTAL":lambda x: x['adresseCodePostalLieu'],
    "THEME":lambda x: extract_info(data, "themes", ["MUSIQUE", "ART", "SCIENCE", "SPECTACLE", "LIVRE"]),
    "SUBTHEME":lambda x: extract_info(data, "themes", subtheme),
    "TYPE":lambda x: (extract_info(data, "typeOffre", offres)
                      .replace({"PORTES OUVERTES":"VISITE", "CIRCUIT":"VISITE"})),
    "DUREE": lambda x: ((to_datetime(x.periodeDateFin, format='%d/%m/%Y', errors='coerce')-
                        to_datetime(x.periodeDateDebut, format='%d/%m/%Y', errors='coerce'))
                        .dt.days.clip(1, 2*365))
})

data = data[['REGION', 'COMMUNE', 'THEME', 'SUBTHEME', 'TYPE', 'DUREE', 'CODE_POSTAL']]

data = data.merge(pop, on='CODE_POSTAL')

,REGION,COMMUNE,THEME,SUBTHEME,TYPE,DUREE,CODE_POSTAL,POPUlATION
0,Bourgogne,Louhans,ART,RAP,MUSÉE,NaN,71500,123.0
1,Bourgogne,Louhans,ART,RAP,VISITE,347.0,71500,123.0
2,Bourgogne,Louhans,NaN,NaN,EXPOSITION,90.0,71500,123.0
3,Bourgogne,Louhans,NaN,NaN,EXPOSITION,156.0,71500,123.0
4,Bourgogne,La Chapelle-Naude,SPECTACLE,THÉÂTRE,SPECTACLE,1.0,71500,123.0
5,Bourgogne,La Chapelle-Naude,SPECTACLE,THÉÂTRE,SPECTACLE,1.0,71500,123.0
6,Bourgogne,La Chapelle Naude,MUSIQUE,FOLKLORIQUES OU TRADITIONNELLES,FESTIVAL,2.0,71500,123.0
7,Bourgogne,Verdun-sur-le-Doubs,SCIENCE,NaN,MUSÉE,NaN,71350,903.0
8,Bretagne,Rennes,ART,ART CONTEMPORAIN,EXPOSITION,30.0,35011,297.0
9,Rhône-Alpes,La Côte-Saint-Andre,MUSIQUE,CLASSIQUE ET CONTEMPORAINE,FESTIVAL,10.0,38260,503.0


In [106]:
data

,REGION,COMMUNE,THEME,SUBTHEME,TYPE,DUREE,CODE_POSTAL
0,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,SPECTACLE,12.0,93200
1,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,SPECTACLE,8.0,93200
2,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,SPECTACLE,1.0,93200
3,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,FESTIVAL,12.0,93200
4,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,SPECTACLE,3.0,93200
5,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,SPECTACLE,8.0,93200
6,Ile-de-France,Saint-Denis,SPECTACLE,CIRQUE,SPECTACLE,21.0,93200
7,Bourgogne,Louhans,ART,RAP,MUSÉE,NaN,71500
8,Bourgogne,Verdun-sur-le-Doubs,SCIENCE,NaN,MUSÉE,NaN,71350
9,Ile-de-France,Milly-la-Forêt,NaN,NaN,EXPOSITION,1.0,91490
